本文主要介绍了模型训练中的各方面内容，包括模型构建、loss、优化器、metrics、正则化、学习率、激活函数、epochs、参数初始化、超参数搜索等。

In [9]:
import tensorflow as tf
from tensorflow import keras
import sklearn
import numpy as np
import pandas
import matplotlib as mpl
print(tf.__version__)

2.5.0



## 5、超参数搜索

神经网络的灵活性也是它们的主要缺点之一：有许多需要调整的超参数。你不仅可以使用任何可以想象的网络结构，而且即使在简单的MLP中，你也可以更改层数、每层神经元数、每层要使用的激活函数的类型、权重初始化逻辑，以及更多。

一种选择是简单地尝试超参数的许多组合，然后查看哪种对验证集最有效（或使用K折交叉验证）。例如我们可以像第2章中一样使用GridSearchCV或RandomizedSearchCV来探索超参数空间。为此我们需要将Keras模型包装在模仿常规ScikitLearn回归器的对象中。

下面我们详细介绍在tensorflow中进行超参数搜索的方式。


我们先构建一个基本模型用于之后的超参数搜索。本次我们使用housing数据集。


In [10]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
x_train = scaler.fit_transform(X_train)
x_valid = scaler.transform(X_valid)
x_test = scaler.transform(X_test)




超参数搜索，一种选择是简单地尝试超参数的许多组合，然后查看哪种对验证集最有效（或使用K折交叉验证）。例如我们可以使用GridSearchCV或RandomizedSearchCV来探索超参数空间。为此我们需要将Keras模型包装在模仿常规ScikitLearn回归器的对象中。第一步是创建一个函数，该函数将在给定一组超参数的情况下构建并编译Keras模型：

In [11]:
def build_model(n_hidden=1, n_neurons=10, learning_rate=0.0001, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics='mse')
    return model



我们简单看一下只运行一次模型的情况。

指定任何超参数，因此它将使用我们在build_model（）中定义的默认超参数。现在，我们可以像常规ScikitLearn回归器一样使用该对象：我们可以使用其fit（）方法进行训练，然后使用其score（）方法进行评估，然后使用predict()方法预测。

传递给fit（）方法的任何其他参数都将传递给内部的Keras模型。还要注意，该分数将与MSE相反，因为ScikitLearn希望获得分数，而不是损失（即分数越高越好）。

In [12]:

keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)
keras_reg.fit(x_train, y_train, epochs=5,
             validation_data=(x_valid, y_valid),
             callbacks=[keras.callbacks.EarlyStopping(patience=5)])
mse_test = keras_reg.score(x_test, y_test)
y_pred = keras_reg.predict(x_test[:5])
print(mse_test)
print(y_pred)

Epoch 1/5
363/363 [==============================] - 1s 1ms/step - loss: 5.0193 - mse: 5.0193 - val_loss: 3.9611 - val_mse: 3.9611
Epoch 2/5
363/363 [==============================] - 0s 766us/step - loss: 3.5299 - mse: 3.5299 - val_loss: 3.1452 - val_mse: 3.1452
Epoch 3/5
363/363 [==============================] - 0s 879us/step - loss: 2.6217 - mse: 2.6217 - val_loss: 2.8003 - val_mse: 2.8003
Epoch 4/5
363/363 [==============================] - 0s 798us/step - loss: 2.0493 - mse: 2.0493 - val_loss: 2.5640 - val_mse: 2.5640
Epoch 5/5
162/162 [==============================] - 0s 507us/step - loss: 1.5176 - mse: 1.5176
-1.5176007747650146
[0.76028025 0.8603613  1.949896   1.5127833  1.2329737 ]


下面我们开始超参数搜索。

我们不想训练和评估这样的单个模型，尽管我们想训练数百个变体，并查看哪种变体在验证集上表现最佳。由于存在许多超参数，因此最好使用随机搜索而不是网格搜索。让我们尝试探索隐藏层的数量、神经元的数量和学习率：


In [13]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(10, 30),
    "learning_rate": reciprocal(3e-4, 3e-3),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(x_train, x_train, epochs=5,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=3)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/5
242/242 [==============================] - 1s 1ms/step - loss: 1.2812 - mse: 1.2812 - val_loss: 271016.0938 - val_mse: 271016.0938
Epoch 2/5
242/242 [==============================] - 0s 818us/step - loss: 0.9913 - mse: 0.9913 - val_loss: 110464.4844 - val_mse: 110464.4844
Epoch 3/5
242/242 [==============================] - 0s 988us/step - loss: 0.9683 - mse: 0.9683 - val_loss: 72900.2344 - val_mse: 72900.2344
Epoch 4/5
242/242 [==============================] - 0s 1ms/step - loss: 0.9603 - mse: 0.9603 - val_loss: 59062.6641 - val_mse: 59062.6641
Epoch 5/5
121/121 [==============================] - 0s 804us/step - loss: 0.8883 - mse: 0.8883
[CV] END learning_rate=0.0022068903028043606, n_hidden=0, n_neurons=14; total time=   1.6s
Epoch 1/5
242/242 [==============================] - 0s 1ms/step - loss: 1.2686 - mse: 1.2686 - val_loss: 2363.8618 - val_mse: 2363.8618
Epoch 2/5
242/242 [==============================] 

RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fc2e648ab80>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc2d86e72e0>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29])},
                   verbose=2)

搜索可能持续数小时，具体时间取决于硬件、数据集的大小、模型的复杂性以及n_iter和cv的值。当结束时，你可以访问找到的最佳参数、最佳分数和经过训练的Keras模型：

In [14]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)
model = rnd_search_cv.best_estimator_.model

{'learning_rate': 0.0023308034965341855, 'n_hidden': 1, 'n_neurons': 21}
-0.9219454526901245


完整代码如下：

In [15]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
x_train = scaler.fit_transform(X_train)
x_valid = scaler.transform(X_valid)
x_test = scaler.transform(X_test) 

def build_model(n_hidden=1, n_neurons=10, learning_rate=0.0001, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics='mse')
    return model

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(10, 30),
    "learning_rate": reciprocal(3e-4, 3e-3),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(x_train, x_train, epochs=5,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=3)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/5
242/242 [==============================] - 1s 1ms/step - loss: 1.1251 - mse: 1.1251 - val_loss: 47374.3945 - val_mse: 47374.3945
Epoch 2/5
242/242 [==============================] - 0s 863us/step - loss: 1.0586 - mse: 1.0586 - val_loss: 81092.0469 - val_mse: 81092.0469
Epoch 3/5
242/242 [==============================] - 0s 876us/step - loss: 1.0360 - mse: 1.0360 - val_loss: 45819.7070 - val_mse: 45819.7070
Epoch 4/5
242/242 [==============================] - 0s 868us/step - loss: 1.0653 - mse: 1.0653 - val_loss: 93077.3594 - val_mse: 93077.3594
Epoch 5/5
121/121 [==============================] - 0s 1ms/step - loss: 0.8770 - mse: 0.8770
[CV] END learning_rate=0.0017487067052027632, n_hidden=1, n_neurons=18; total time=   1.7s
Epoch 1/5
242/242 [==============================] - 1s 2ms/step - loss: 0.9969 - mse: 0.9969 - val_loss: 96573.0547 - val_mse: 96573.0547
Epoch 2/5
242/242 [==============================] - 

RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fc2e648ab80>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc2e403f850>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29])},
                   verbose=2)